# Exploratory Data Analysis

## Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from rapidfuzz import fuzz, process
import unidecode
import requests
import json

In [2]:
sys.path.append(os.path.abspath("../src"))
from consultacep import consulta_cep

## Raw Data

In [3]:
df_customers = pd.read_csv("../data/raw/olist_customers_dataset.csv")
df_order_items = pd.read_csv("../data/raw/olist_order_items_dataset.csv")
df_order_payments = pd.read_csv("../data/raw/olist_order_payments_dataset.csv")
df_orders = pd.read_csv("../data/raw/olist_orders_dataset.csv")
df_products = pd.read_csv("../data/raw/olist_products_dataset.csv")
df_sellers = pd.read_csv("../data/raw/olist_sellers_dataset.csv")
cdf_cities = pd.read_csv("../data/raw/brazilian_cities.csv")
cdf_ceps = pd.read_csv("../data/raw/CEPs_Brasil.csv")


## Checking the relevant data

### Dataframe: Customers

In [4]:
df_customers.sample(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
71460,32f7e622cfbdd94691a85692f992a9d5,bf7bf802783e8b525e33fde224781e98,9210,santo andre,SP
55848,b129c5e7449bfea86f4bef20e296c0ce,6234bff8b25bd8336455555eb5f749da,29934,sao mateus,ES
55453,de5127e844f7bc6e9593315e778d7da1,b868689dabda36770f6ef2b8c39bf8a6,2220,sao paulo,SP
68840,f0688a7a0aa2400361ea273a4872db2d,2dbad1ecf3e38836eacff9531adc5b4a,13273,valinhos,SP
14161,be8c14c16a4d47194ccdfe10f1fc5b1a,c86a25b8f5f6c203bb3471553bdc3200,13157,cosmopolis,SP
13328,be24ef8018f62f3b54f287c7e87d0abb,c2e11be7617544391dfa3905b0c637d6,17065,bauru,SP
61078,2a06665a19465bcdd512dcb3444c1de4,0404ab99cfcbae7ad6cdc6e93eac6641,27264,volta redonda,RJ
83526,46a4ac4dcaf4f17c44f5d7c5a5afaf86,ae8ba97274b08fe1156b1d5d3ad32983,13825,holambra,SP
68688,13e56bf1c485ef13438118c361f92527,d6c6b796206305b682a65170c45d26e3,28625,nova friburgo,RJ
8518,46ebdabde58535b8751aaf7e6f21fe27,ff9c030fb4209fbf442ae1368a6b376a,85905,toledo,PR


In [5]:
df_customers.info()
df_customers.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [6]:
df_customers.duplicated().sum()

np.int64(0)

- df_customers doesn't have duplicate entrys, no columns to drop at first and no Nulls

### Dataframe: Order_items

In [7]:
df_order_items.sample(10)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
79017,b3bab804c76ad164c8aba19684f85e38,1,629e019a6f298a83aeecc7877964f935,c003204e1ab016dfa150abc119207b24,2018-01-05 02:08:47,109.90,14.37
64074,925e0efae10a47529fb9c78338b66d8c,1,d4a3a15f99c94c6741d16d3375b928e6,6560211a19b47992c3666cc44a7e94c0,2017-03-23 14:59:36,49.00,14.11
4173,0964250cecc3ab90b45138c9cbbf7b91,1,a0b7b660f3ff79836867de402e158770,4978a02ea9c8be7a9b5480680f40334e,2018-05-14 23:51:05,119.99,14.93
93513,d3f5cac5ca660216eb21e93d54e9e384,1,803f77475e1b51b47f1bfec4f2ec353f,c9c7905cffc4ef9ff9f113554423e671,2018-07-25 10:32:03,79.99,8.72
1883,044223df7cb37509bfeb2a0df1250ca4,1,b2d98b6f547c14e29a9e72b8e630d936,004c9cd9d87a3c30c522c48c4fc07416,2017-11-08 09:35:30,115.00,21.30
34959,4f22b45861402b6db26c58fd211617cf,1,2dbed858ef1a666ec3afae6566400a06,5b67882648febfb6e1225e6ad7164f6b,2018-03-16 15:15:41,519.99,17.72
46176,68d8838c90191c81a8e3e1273e230414,1,c835fd9d2e466148ac3755300628e33d,a3a38f4affed601eb87a97788c949667,2018-04-25 19:11:00,59.90,23.35
52191,76c5d675f926836b4ff285bff2d38766,1,02a97df83a8a100c7d2b14a02aea6a3d,620c87c171fb2a6dd6e8bb4dec959fc6,2018-07-19 03:04:21,69.90,22.42
33736,4c5e883786df8d555be4dc12f11d3fe7,1,615ee56e97eef79c2b855111d9434dcf,a416b6a846a11724393025641d4edd5e,2018-06-18 04:57:32,112.83,9.14
74316,a95612c7311ad95c69b272aaa4daa6e5,1,9328c3a5c42ae7b32d53e419f9b93d66,cca3071e3e9bb7d12640c9fbe2301306,2017-03-21 04:10:20,50.90,17.20


In [8]:
df_order_items.info()
df_order_items.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [9]:
df_order_items.duplicated().sum()

np.int64(0)

In [10]:
df_order_items.drop(columns=['shipping_limit_date', 'freight_value'], inplace=True)

- df_order_items doesn't have duplicate entrys and Nulls.
- dropping columns: shipping_limit_date, freight_value

### Dataframe: Order_payments

In [11]:
df_order_payments.sample(10)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
17628,1bf8b5f1e3bc76dcaa6414b170ef3ae0,1,boleto,1,143.27
8183,a3e07c6b717e74f8c384d148d7903e66,1,credit_card,1,81.78
14815,c12a78eb30abe88570f3c2c79bea1334,1,credit_card,1,87.30
67818,502b576f4d6a7091842c83677e13013d,1,credit_card,1,152.62
82939,ee27a6c585edd9ed8b99e272587bf04c,1,credit_card,5,57.38
100868,b0707b7956935b194cfa319dc1501d96,1,credit_card,4,46.22
84839,e39d427402fa3f754c7fd38ceb788901,1,credit_card,6,95.80
40585,f68bf4a4d690be2a9ba9402b0862ad34,1,credit_card,1,166.68
25826,cbf5771b59c1f5688a8f8224f08607eb,1,credit_card,10,239.42
32766,60f6dd1c9030b6132080d65599736d7f,1,credit_card,2,43.87


In [12]:
df_order_payments.info()
df_order_payments.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

In [13]:
df_order_payments.duplicated().sum()

np.int64(0)

In [14]:
df_order_payments.drop(columns=['payment_sequential','payment_installments','payment_type'], inplace=True)

- df_order_payments doesn't have duplicate entrys and Nulls.
- dropping columns: payment_sequential, payment_type, payment_installments

### Dataframe: Orders

In [15]:
df_orders.sample(10)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
3362,03e94224875e6dec154cc7a142ce5de2,bb7651b2df4512e6c29ff35a7b8cd754,delivered,2018-04-23 10:09:03,2018-04-24 18:11:03,2018-04-24 17:42:08,2018-04-25 18:58:59,2018-05-08 00:00:00
71325,06c82be9b8551eb37a9b8e7e05dec631,226a2de9b9feb22a76582963ebcf1e96,delivered,2017-07-11 14:27:14,2017-07-11 14:43:41,2017-07-26 03:19:36,2017-08-01 16:17:38,2017-08-02 00:00:00
45740,970b0850a5b6ccaa34a0e887e8d10a5b,4619dca3ccf1e57f50ba85089e2370f6,delivered,2018-03-17 11:06:27,2018-03-19 20:28:03,2018-03-22 21:32:46,2018-04-02 13:26:28,2018-04-05 00:00:00
89929,fb01e8d8215d266abffb5ca379063244,0d7b6a46b67046fa56193773a361e7ac,delivered,2017-04-03 22:00:05,2017-04-03 22:15:14,2017-04-13 19:07:15,2017-04-19 09:53:18,2017-05-02 00:00:00
16942,cb7c837f8c5240816176dc115c21a746,d60805f01a84d8b2d5acae981e3c770f,delivered,2018-06-21 20:27:51,2018-06-26 04:58:46,2018-06-27 10:28:00,2018-07-04 19:04:37,2018-07-27 00:00:00
2079,ed98c37d860890f940e2acd83629fdd1,71f8cd490677ee95362201588c0eaff4,delivered,2017-12-15 12:57:19,2017-12-15 13:13:05,2017-12-19 22:57:44,2017-12-22 01:28:07,2018-01-05 00:00:00
116,4b3a605942f29d490cb74bd6ace6b9f0,e8a332c3433fbd37920ec89fc295d075,delivered,2017-09-29 09:32:17,2017-09-29 09:44:12,2017-09-29 21:58:48,2017-09-30 16:55:39,2017-10-17 00:00:00
94696,72b9d86fe4e984d9879013b9ecbb25db,8b122c119e36ca9dd381d2bfc02112ea,delivered,2018-04-11 22:29:32,2018-04-13 12:35:27,2018-04-13 21:26:48,2018-04-23 18:41:58,2018-05-23 00:00:00
26173,103e18e9325ee41d8cfa4902f8abc16f,aa28fcb213db1415749a665f82704181,delivered,2018-01-14 14:34:55,2018-01-14 14:48:41,2018-01-17 22:16:57,2018-01-24 21:54:49,2018-02-05 00:00:00
21659,ffc29f864fd689fab9ae81be82baf8ef,77d2c30e1dff5d4440ec316f8a2ef2a2,delivered,2017-07-30 20:33:10,2017-07-31 20:43:11,2017-08-04 17:45:04,2017-08-09 19:56:29,2017-08-24 00:00:00


In [16]:
df_orders.info()
df_orders.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [17]:
df_orders.duplicated().sum()

np.int64(0)

In [18]:
df_orders.drop(columns=['order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date'], inplace=True)

In [19]:
df_orders.info()
df_orders.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   order_id                  99441 non-null  object
 1   customer_id               99441 non-null  object
 2   order_status              99441 non-null  object
 3   order_purchase_timestamp  99441 non-null  object
dtypes: object(4)
memory usage: 3.0+ MB


order_id                    0
customer_id                 0
order_status                0
order_purchase_timestamp    0
dtype: int64

- df_order_payments doesn't have duplicate entrys and Nulls on relevant columns.
- dropping columns: order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date

### Dataframe: Products

In [20]:
df_products.sample(10)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
16454,19290ebcf78e385cd9fcd48ce5e1393d,ferramentas_jardim,41.0,320.0,1.0,267.0,27.0,13.0,17.0
5647,207bb2d8180c2c7654872f6fab96e40e,telefonia,30.0,305.0,1.0,100.0,19.0,3.0,11.0
7643,88596b69f04745db1f8ba135da5c708d,papelaria,59.0,296.0,1.0,310.0,21.0,7.0,15.0
7636,db1154ea7d1c50d8a5939a9f7fbc3b2a,brinquedos,42.0,283.0,1.0,5200.0,50.0,10.0,50.0
6985,bdd59148fd23281bb706539815e39bf3,perfumaria,57.0,598.0,1.0,450.0,18.0,18.0,18.0
30885,422fe5f2947a7aa4b72de6e4e41f543f,perfumaria,58.0,598.0,2.0,700.0,43.0,16.0,27.0
16093,efeb7b23db334d617250aca97cd3f1e8,cama_mesa_banho,51.0,404.0,1.0,1500.0,30.0,15.0,20.0
15130,988117394d514cb9fb1b367714159a67,ferramentas_jardim,58.0,1378.0,1.0,3500.0,21.0,23.0,11.0
21021,50e1b118f266b655f613f753914c42db,cama_mesa_banho,44.0,630.0,1.0,800.0,16.0,20.0,28.0
695,1ecaa47c3ade7f098ca5dd97e8dcb018,telefonia,54.0,653.0,1.0,150.0,19.0,3.0,11.0


In [21]:
df_products.info()
df_products.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [22]:
df_products.duplicated().sum()

np.int64(0)

In [23]:

df_products.drop(columns= ['product_name_lenght','product_description_lenght',
    'product_photos_qty','product_weight_g','product_length_cm','product_height_cm',
    'product_width_cm'], inplace=True)

In [24]:
df_products['product_category_name'] = df_products['product_category_name'].fillna('sem categoria')

In [25]:
df_products.info()
df_products.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   product_id             32951 non-null  object
 1   product_category_name  32951 non-null  object
dtypes: object(2)
memory usage: 515.0+ KB


product_id               0
product_category_name    0
dtype: int64

- df_products doesn't have duplicate entrys.
- Null cells in product_category_name filled with placeholder 'sem categoria'
- dropping columns: 'product_name_lenght','product_description_lenght','product_photos_qty','product_weight_g','product_length_cm','product_height_cm','product_width_cm'

### Dataframe: Sellers

In [26]:
df_sellers.sample(10)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
3076,9e6967d3cf386d284251784b18ccb485,9550,sao caetano do sul,SP
2848,3f84fefc07dc159b268d4a77d3e8c425,83833,fazenda rio grande,PR
352,2dd1fb6f0e42a2243b8a01e40cd41216,80240,curitiba,PR
2218,bbaff50f3b708fda865918715276cd87,12940,atibaia,SP
966,fe19dce63ae80346207c6c55713d1023,38010,uberaba,MG
2001,85d9eb9ddc5d00ca9336a2219c97bb13,31255,belo horizonte,MG
694,385799a1cddbd26dfeb9157356a42282,4676,sao paulo,SP
765,ba90964cff9b9e0e6f32b23b82465f7b,7140,guarulhos,SP
1231,4b98b0428fc1932cc2514ef9ce2505e6,6795,taboao da serra,SP
760,bc503e244f187939d10c92a5c8e378ce,16200,birigui,SP


In [27]:
df_sellers.info()
df_sellers.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

In [28]:
df_sellers.duplicated().sum()

np.int64(0)

- df_sellers doesn't have duplicate entrys, no columns to drop at first and no Nulls

## Data Conversion and normalization

### Dataframe: Products

In [29]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   product_id             32951 non-null  object
 1   product_category_name  32951 non-null  object
dtypes: object(2)
memory usage: 515.0+ KB


- Normalizing to a readable entries

In [30]:
df_products['product_category_name'] = (
    df_products['product_category_name']
    .str.replace('_', ' ')      
    .str.title()                
    .str.strip()    
)

In [31]:
sorted(df_products['product_category_name'].dropna().unique())

['Agro Industria E Comercio',
 'Alimentos',
 'Alimentos Bebidas',
 'Artes',
 'Artes E Artesanato',
 'Artigos De Festas',
 'Artigos De Natal',
 'Audio',
 'Automotivo',
 'Bebes',
 'Bebidas',
 'Beleza Saude',
 'Brinquedos',
 'Cama Mesa Banho',
 'Casa Conforto',
 'Casa Conforto 2',
 'Casa Construcao',
 'Cds Dvds Musicais',
 'Cine Foto',
 'Climatizacao',
 'Consoles Games',
 'Construcao Ferramentas Construcao',
 'Construcao Ferramentas Ferramentas',
 'Construcao Ferramentas Iluminacao',
 'Construcao Ferramentas Jardim',
 'Construcao Ferramentas Seguranca',
 'Cool Stuff',
 'Dvds Blu Ray',
 'Eletrodomesticos',
 'Eletrodomesticos 2',
 'Eletronicos',
 'Eletroportateis',
 'Esporte Lazer',
 'Fashion Bolsas E Acessorios',
 'Fashion Calcados',
 'Fashion Esporte',
 'Fashion Roupa Feminina',
 'Fashion Roupa Infanto Juvenil',
 'Fashion Roupa Masculina',
 'Fashion Underwear E Moda Praia',
 'Ferramentas Jardim',
 'Flores',
 'Fraldas Higiene',
 'Industria Comercio E Negocios',
 'Informatica Acessorios',
 

In [32]:
manual_corrections = {
    'Agro Industria E Comercio': 'Agro Indústria e Comércio',
    'Artes E Artesanato': 'Artes e Artesanato',
    'Artigos De Festas': 'Artigos de Festas',
    'Artigos De Natal': 'Artigos de Natal',
    'Beleza Saude':'Beleza e Saude',
    'Cama Mesa Banho': 'Cama Mesa e Banho',
    'Casa Conforto': 'Casa e Conforto',
    'Casa Conforto 2': 'Casa e Conforto',
    'Casa Construcao': 'Casa e Construção',
    'Cds Dvds Musicais': 'Cds Dvds e Musicais',
    'Cine Foto': 'Cine e Foto',
    'Climatizacao': 'Climatização',
    'Consoles Games': 'Consoles e Games',
    'Construcao Ferramentas Construcao': 'Artigos de Construção',
    'Construcao Ferramentas Ferramentas': 'Artigos de Construção',
    'Construcao Ferramentas Iluminacao': 'Artigos de Construção',
    'Construcao Ferramentas Jardim': 'Artigos de Construção',
    'Construcao Ferramentas Seguranca': 'Artigos de Construção',
    'Dvds Blu Ray': 'Dvds/Blu-Ray',
    'Eletrodomesticos': 'Eletrodomésticos',
    'Eletrodomesticos 2': 'Eletrodomésticos',
    'Eletronicos': 'Eletrônicos',
    'Eletroportateis': 'Eletroportáteis',
    'Esporte Lazer': 'Esporte e Lazer',
    'Fashion Bolsas E Acessorios': 'Bolsas e Acessórios',
    'Fashion Calcados':'Calçados',
    'Fashion Esporte': 'Moda Esportiva',
    'Fashion Roupa Feminina': 'Roupa Feminina',
    'Fashion Roupa Infanto Juvenil': 'Infanto Juvenil',
    'Fashion Roupa Masculina': 'Roupa Masculina',
    'Fashion Underwear E Moda Praia': 'Underwear e Moda Praia',
    'Ferramentas Jardim': 'Ferramentas de Jardinagem',
    'Fraldas Higiene': 'Fraldas e Higiene',
    'Industria Comercio E Negocios': 'Indústria Comércio e Negócios',
    'Informatica Acessorios': 'Acessórios de Informática',
    'Livros Importados': 'Livros',
    'Livros Interesse Geral': 'Livros',
    'Livros Tecnicos': 'Livros',
    'Malas Acessorios': 'Malas e Acessórios',
    'Moveis Colchao E Estofado': 'Móveis',
    'Moveis Cozinha Area De Servico Jantar E Jardim': 'Móveis',
    'Moveis Decoracao': 'Móveis',
    'Moveis Escritorio': 'Móveis',
    'Moveis Quarto': 'Móveis',
    'Moveis Sala': 'Móveis',
    'Musica': 'Música',
    'Portateis Casa Forno E Cafe': 'Casa Forno e Café',
    'Portateis Cozinha E Preparadores De Alimentos': 'Cozinha e Preparadores de Alimentos',
    'Relogios Presentes': 'Relógios e Presentes',
    'Seguros E Servicos':'Seguros e Serviços',
    'Sinalizacao E Seguranca': 'Sinalização e Segurança',
    'Tablets Impressao Imagem': 'Tablets Impressão e Imagem',
    'Utilidades Domesticas': 'Utilidades Domésticas'
}
df_products['product_category_name'] = df_products['product_category_name'].replace(manual_corrections)

### Dataframe: Orders

In [33]:
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  99441 non-null  object        
 1   customer_id               99441 non-null  object        
 2   order_status              99441 non-null  object        
 3   order_purchase_timestamp  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 3.0+ MB


### Dataframe: Order_items and Order_Payments

In [34]:
df_order_items.info() #ok
df_order_payments.info() #ok


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   order_id       112650 non-null  object 
 1   order_item_id  112650 non-null  int64  
 2   product_id     112650 non-null  object 
 3   seller_id      112650 non-null  object 
 4   price          112650 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   order_id       103886 non-null  object 
 1   payment_value  103886 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.6+ MB


### Dataframe: Customer and Sellers

In [35]:
df_customers.info()
df_sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8

- Cities in df_customers and df_sellers has some strange entries
- Using a dataset of Brazilian ZIP codes to normalize both dataframes.

In [42]:
def buscar_local_por_cep(cep, cdf):
    cdf_temp = cdf.copy()
    cdf_temp['CEP Inicial'] = (cdf_temp['CEP Inicial'] // 1000).astype(int)
    cdf_temp['CEP Final'] = (cdf_temp['CEP Final'] // 1000).astype(int)
    
    row = cdf_temp[(cdf_temp['CEP Inicial'] <= cep) & (cdf_temp['CEP Final'] >= cep)]

    if not row.empty:
        local = row.iloc[0]['Localidade']
        estado = row.iloc[0]['Estado']  # ou 'estado', se já estiver nome completo
        return pd.Series({'cidade_corrigida': local, 'estado_corrigido': estado})
    else:
        return pd.Series({'cidade_corrigida': None, 'estado_corrigido': None}) 
    

df_customers[['cidade_corrigida', 'estado_corrigido']] = (
    df_customers['customer_zip_code_prefix'].apply(lambda cep: buscar_local_por_cep(cep, cdf_ceps))
)

df_sellers[['cidade_corrigida', 'estado_corrigido']] = (
    df_sellers['seller_zip_code_prefix'].apply(lambda cep: buscar_local_por_cep(cep, cdf_ceps))
)



df_customers[df_customers['cidade_corrigida'].isna()]
df_customers[df_customers['estado_corrigido'].isna()]
df_sellers[df_sellers['cidade_corrigida'].isna()]
df_sellers[df_sellers['estado_corrigido'].isna()]

,seller_id,seller_zip_code_prefix,seller_city,seller_state,cidade_corrigida,estado_corrigido


> Running a function to check for the CEP and return the city and state

In [43]:
df_customers[df_customers['estado_corrigido'].isna()]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,cidade_corrigida,estado_corrigido
2262,fe196aba95e52d084e7bb60dbd8a23f1,93b4c8bdafa9877757c77b519415d274,89130,indaial,SC,None,None
5525,81eaa0c0dfb54f5f9d7d11734f162e65,b0369cc51e0e1618a76c7174132b93cc,89130,indaial,SC,None,None
5648,a059473225838f7005cf82e2f6b9a18d,1df28da6a384e9822da017c6d44b0656,89130,indaial,SC,None,None
11226,4560c6d8bfdd3e61555dca85d900e1e6,ad054a269e8691d8e2727d1611f3950e,89130,indaial,SC,None,None
16766,110d797e5a5055538df046c2cad8f3d4,1a4d4db5393b394803df246ca68872b4,98900,santa rosa,RS,None,None
30064,988126b4ddf725d9724e4318872ea2ae,507dc9becd4fc65635d90682dfa9d3a3,98900,santa rosa,RS,None,None
33614,7775deebb59b9257b62fd7f3117efa10,26d23e38ed702dc7a476dbb107192848,89130,indaial,SC,None,None
44046,6536b7e35e2598371da42ea384a43ddb,760aa83faf9321a80f455369d7339a0a,89130,indaial,SC,None,None
60564,ff69fe294d9da2e683dede2ddb70b6ae,cd6ff01588442a00ebf2e3829d88e819,98900,santa rosa,RS,None,None
69067,12217655416d9edb5594c7dc6597b85f,215d7119ee04ba8fdd85407e222b90da,89130,indaial,SC,None,None


In [44]:
df_sellers[df_sellers['cidade_corrigida'].isna()]

,seller_id,seller_zip_code_prefix,seller_city,seller_state,cidade_corrigida,estado_corrigido


> Df sellers got all correct, but some missing values on df_customers

In [ ]:
#df_customers[df_customers['estado_corrigido'].isna()]
df_customers.loc[(df_customers['customer_city'] == 'indaial') & (df_customers['cidade_corrigida'].isna()), 'cidade_corrigida'] = "Indaial"
df_customers.loc[(df_customers['customer_city'] == 'indaial') & (df_customers['estado_corrigido'].isna()), 'estado_corrigido'] = "Santa Catarina"
df_customers.loc[(df_customers['customer_city'] == 'santa rosa') & (df_customers['cidade_corrigida'].isna()), 'cidade_corrigida'] = "Santa Rosa"
df_customers.loc[(df_customers['customer_city'] == 'santa rosa') & (df_customers['estado_corrigido'].isna()), 'estado_corrigido'] = "Rio Grande do Sul"

> Manually inserting the right values.

In [ ]:

df_customers[df_customers['estado_corrigido'].isna()]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,cidade_corrigida,estado_corrigido


- Saving cleaned data

In [59]:
df_customers.to_csv("../data/cleaned/df_customers.csv", index=False)
df_order_items.to_csv("../data/cleaned/df_order_items.csv", index=False)
df_order_payments.to_csv("../data/cleaned/df_order_payments.csv", index=False)
df_orders.to_csv("../data/cleaned/df_orders.csv", index=False)
df_products.to_csv("../data/cleaned/df_products.csv", index=False)
df_sellers.to_csv("../data/cleaned/df_sellers.csv", index=False)


## Derived Data

In [ ]:
df_sellers.rename(columns={
    'cidade_corrigida':'seller_city_clean',
    'estado_corrigido':'seller_state_clean'
    },inplace=True)

df_customers.rename(columns={
    'cidade_corrigida':'customer_city_clean',
    'estado_corrigido':'customer_state_clean'
    },inplace=True)

- Creating the final sales dataframe by merging the cleaned dataframes.

In [ ]:
#df_order_items.sort_values(["order_id", "order_item_id"],ascending=[1,1])
proc_df_sales = df_order_items.merge(
    df_products, on='product_id', how='left').merge(
        df_sellers[['seller_id','seller_city_clean','seller_state_clean']], on='seller_id',how='left')

> Merged df_order_items with products and sellers, at first.

In [ ]:
proc_order_by_customer = df_orders.merge(df_customers[['customer_id','customer_unique_id','customer_city_clean','customer_state_clean']],on='customer_id',how='left')
proc_order_by_customer = proc_order_by_customer.loc[proc_order_by_customer['order_status'] == 'delivered']
proc_order_by_customer.drop(columns=['order_status'],inplace=True) 

> Merged the orders with the customer informations

In [ ]:
 proc_df_sales[~proc_df_sales['order_id'].isin(proc_order_by_customer['order_id'])]

,order_id,order_item_id,product_id,seller_id,price,product_category_name,seller_city_clean,seller_state_clean
79,002f19a65a2ddd70a090297872e6d64e,1,9eae06d51aaa383b2bed5547a19d581c,1835b56ce799e6a4dc4eddc053f04066,53.98,Cama Mesa e Banho,Ibitinga,São Paulo
84,00310b0c75bb13015ec4d82d341865a4,1,c8e7c2ef329fcda4a233e7e2f8bb8b7d,a2deecd5398f5df4987110c80a1972a3,39.90,Utilidades Domésticas,São Paulo,São Paulo
260,00a99c50fdff7e36262caba33821875a,1,76d75f398634bf194ba99d7297e7d1fa,974cf2cb8f4b7add98709c30df02fe10,52.99,Acessórios de Informática,Araraquara,São Paulo
270,00ae7a8b4936674ebb701d4a23719a79,1,66f09a6de4f2ef63aaf5349b60a0e0c0,bf0d50a6410d487dc97d2baac0a8c0be,27.90,Automotivo,Piracicaba,São Paulo
422,00ff0cf5583758e6964723e42f111bf4,1,bb1fe952bdc16ef8974d4ca11aec19b7,b17b679f4f5ce2e03ce6968c62648246,154.90,Beleza e Saude,Ribeirão Preto,São Paulo
...,...,...,...,...,...,...,...,...
112211,feebdfdb4759bd4600d52547c81b4fa8,1,0c7536abc1af51a81fa806097dfe3597,7d76b645482be4a332374e8223836592,24.90,Telefonia,São Paulo,São Paulo
112256,ff0768e3356919b32e3d7f30baecfcfb,1,b27653c33039497c46ceba21db46a43c,3e8bd881d92466e63ba9e3e809cd3c50,379.90,Acessórios de Informática,Rio de Janeiro,Rio de Janeiro
112340,ff358345576214cc9e08488188973c84,1,6803077179d24889430188e03fafd31a,128639473a139ac0f3e5f5ade55873a5,19.90,Eletrônicos,Maringá,Paraná
112384,ff536d93ae4214b4d51c2894ccfc569f,1,53d2c16e7356d124bb2a3f64ca920754,4e922959ae960d389249c378d1c939f5,39.00,Esporte e Lazer,Jacareí,São Paulo


> This missing indexes mean they belong a non delivered order, therefore can't be accounted on the KPI's.


In [ ]:
proc_df_sales = proc_df_sales[proc_df_sales['order_id'].isin(proc_order_by_customer['order_id'])]
proc_df_sales.reset_index(drop=True, inplace=True)

In [85]:
total_spent_by_order = proc_df_sales.groupby('order_id')['price'].sum().reset_index()
proc_order_by_customer = proc_order_by_customer.merge(total_spent_by_order,on='order_id',how='left')

- Generating a one dataframe that can be extracted most of our measures.

In [ ]:
proc_df_sales = proc_df_sales.merge(proc_order_by_customer[['order_id','order_purchase_timestamp','customer_unique_id','customer_city_clean','customer_state_clean']],on='order_id',how='left')
proc_df_sales

,order_id,order_item_id,product_id,seller_id,price,product_category_name,seller_city_clean,seller_state_clean,order_purchase_timestamp,customer_unique_id,customer_city_clean,customer_state_clean
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,Cool Stuff,Volta Redonda,Rio de Janeiro,2017-09-13 08:59:02,871766c5855e863f6eccc05f988b23cb,Campos dos Goytacazes,Rio de Janeiro
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,Pet Shop,São Paulo,São Paulo,2017-04-26 10:53:06,eb28e67c4c0b83846050ddfb8a35d051,Santa Fé do Sul,São Paulo
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,Móveis,Borda da Mata,Minas Gerais,2018-01-14 14:33:31,3818d81c6709e39d06b2738a8d3a2474,Pará de Minas,Minas Gerais
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,Perfumaria,Franca,São Paulo,2018-08-08 10:00:35,af861d436cfc08b2c2ddefd0ba074622,Atibaia,São Paulo
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,Ferramentas de Jardinagem,Loanda,Paraná,2017-02-04 13:57:51,64b576fb70d441e8f1b2d7d446e483c5,Várzea Paulista,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...
110192,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,299.99,Utilidades Domésticas,Itajaí,Santa Catarina,2018-04-23 13:57:06,0c9aeda10a71f369396d0c04dce13a64,São Luís,Maranhão
110193,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,350.00,Acessórios de Informática,São Paulo,São Paulo,2018-07-14 10:26:46,0da9fe112eae0c74d3ba1fe16de0988b,Curitiba,Paraná
110194,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,99.90,Esporte e Lazer,Curitiba,Paraná,2017-10-23 17:07:56,cd79b407828f02fdbba457111c38e4c4,São Paulo,São Paulo
110195,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,55.99,Acessórios de Informática,São Paulo,São Paulo,2017-08-14 23:02:59,eb803377c9315b564bdedad672039306,Vinhedo,São Paulo


- Creating the category x sale relation

In [ ]:
proc_category_sales = proc_df_sales.groupby('product_category_name')['price'].sum().reset_index().sort_values("price", ascending=False)
proc_category_sales.reset_index(drop=True, inplace=True)

- Creating the customer x sales relation

In [124]:
proc_sales_per_customer = proc_df_sales.groupby('order_id')['price'].sum().reset_index().merge(
    df_orders[['order_id','customer_id']],on='order_id',how='left'
).merge(
    df_customers[['customer_id','customer_unique_id']],on='customer_id',how='left'
)
proc_sales_per_customer

,order_id,price,customer_id,customer_unique_id
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,3ce436f183e68e07877b285a838db11a,871766c5855e863f6eccc05f988b23cb
1,00018f77f2f0320c557190d7a144bdd3,239.90,f6dd3ec061db4e3987629fe6b26e5cce,eb28e67c4c0b83846050ddfb8a35d051
2,000229ec398224ef6ca0657da4fc703e,199.00,6489ae5e4333f3693df5ad4372dab6d3,3818d81c6709e39d06b2738a8d3a2474
3,00024acbcdf0a6daa1e931b038114c75,12.99,d4eb9395c8c0431ee92fce09860c5a06,af861d436cfc08b2c2ddefd0ba074622
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,58dbd0b2d70206bf40e62cd34e84d795,64b576fb70d441e8f1b2d7d446e483c5
...,...,...,...,...
96473,fffc94f6ce00a00581880bf54a75a037,299.99,b51593916b4b8e0d6f66f2ae24f2673d,0c9aeda10a71f369396d0c04dce13a64
96474,fffcd46ef2263f404302a634eb57f7eb,350.00,84c5d4fbaf120aae381fad077416eaa0,0da9fe112eae0c74d3ba1fe16de0988b
96475,fffce4705a9662cd70adb13d4a31832d,99.90,29309aa813182aaddc9b259e31b870e6,cd79b407828f02fdbba457111c38e4c4
96476,fffe18544ffabc95dfada21779c9644f,55.99,b5e6afd5a41800fdf401e0272ca74655,eb803377c9315b564bdedad672039306


- Calculating the mean value spent per customer

In [ ]:
proc_sales_per_customer = proc_sales_per_customer.groupby('customer_unique_id', as_index=False).agg({'price': 'sum', 'order_id': 'nunique'})
proc_sales_per_customer.rename(columns={'order_id':'num_de_compras'},inplace=True)
proc_sales_per_customer['ticket_medio'] = proc_sales_per_customer['price'] / proc_sales_per_customer['num_de_compras']
proc_sales_per_customer = proc_sales_per_customer.merge(
    df_customers[['customer_unique_id','customer_state','customer_city_clean','customer_state_clean']],on='customer_unique_id',how='left'
    )
proc_sales_per_customer

,customer_unique_id,price,num_de_compras,ticket_medio,customer_state,customer_city_clean,customer_state_clean
0,0000366f3b9a7992bf8c76cfdf3221e2,129.90,1,129.90,SP,Cajamar,São Paulo
1,0000b849f77a49e4a4ce2b2a4ca5be3f,18.90,1,18.90,SP,Osasco,São Paulo
2,0000f46a3911fa3c0805444483337064,69.00,1,69.00,SC,São José,Santa Catarina
3,0000f6ccb0745a6a4b88665a16c9f078,25.99,1,25.99,PA,Belém,Pará
4,0004aac84e0df4da2b147fca70cf8255,180.00,1,180.00,SP,Sorocaba,São Paulo
...,...,...,...,...,...,...,...
96678,fffcf5a5ff07b0908bd4e2dbc735a684,1570.00,1,1570.00,PE,Sanharó,Pernambuco
96679,fffea47cd6d3cc0a88bd621562a9d061,64.89,1,64.89,BA,Feira de Santana,Bahia
96680,ffff371b4d645b6ecea244b27531430a,89.90,1,89.90,MT,Sinop,Mato Grosso
96681,ffff5962728ec6157033ef9805bacc48,115.00,1,115.00,ES,Bom Jesus do Norte,Espírito Santo


## Export

In [ ]:
proc_df_sales.to_csv("../data/processed/sales.csv", index=False)
proc_category_sales.to_csv("../data/processed/sales_category.csv", index=False)
proc_sales_per_customer.to_csv("../data/processed/sales_per_customer.csv", index=False)